# 1. Install and Import dependencies

In [4]:
import torch
print(torch.cuda.is_available())


True


In [5]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import requests
from bs4 import BeautifulSoup
import re

# 2 Instantiate model

In [7]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 3 Encode and calculate sentiment

In [8]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [9]:
tokens[0]

tensor([  101, 10197, 10140, 12050, 10502, 12296, 10598, 10662, 16197,   119,
        11838,   102])

In [10]:
tokenizer.decode(tokens[0])

'[CLS] it was good but couldve been better. great [SEP]'

In [11]:
result = model(tokens)
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [12]:
int(torch.argmax(result.logits))+1

4

# 4 Collect Reviews

In [ ]:
r = requests.get('https://www.tripadvisor.com/ShowUserReviews-g293936-d1063121-r122503332-Star-Dhaka_City_Dhaka_Division')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*partial_entry.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [49]:
reviews

# 5 Load Reviews into Dataframe and Score

In [30]:
import numpy as np
import pandas as pd

In [32]:
df=pd.DataFrame(np.array(reviews), columns=['reviews'])

In [34]:
df['reviews'].iloc[2]

"i have visited Star Kabab several times (Banani location and Dhanmondi as well). I took carry out - naan, boti kabab, chicken tikka and found them to be tasty, not dry as it most kabab places though I have tasted bbetter chicken karia eslewhere which they cook in a large tawa. But the prices were very reasonable. Once we had breakfast - paratas, egg ommelette, some meat dish, tea, and vegetables and the bill came so low I kept asking the waitter if this was correct. It wasn't even tk 200. Service is fast and i mean fast. Only thing you should keep in mind, its not the ideal setting for dinner dates, just a regular place with a regular no frills decor serving good kababs and etc. at very affordable rates. Can't go wrong with this place."

In [35]:
def sentiment_score(reviews):
    tokens = tokenizer.encode(reviews, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [39]:
sentiment_score(df['reviews'].iloc[1])

5

In [41]:
df['Sentiment'] = df['reviews'].apply(lambda x : sentiment_score(x[:512]))

In [42]:
df

,reviews,Sentiment
0,Yammy.... you will remember your whole life......,5
1,"if u want to have a decent breakfast, a good l...",5
2,i have visited Star Kabab several times (Banan...,3
3,I went there several times when I went to BD. ...,4
4,Food is relly tasty..but da environment is hor...,1
